# Recommender system

### Library import

In [4]:
import pandas as pd

In [5]:
movies = "../lab_data/movies.csv"
ratings = "../lab_data/ratings.csv"

df_movies = pd.read_csv(movies, usecols=["movieId", "title"], dtype={"movieId": "int32", "title": "str"})
df_ratings = pd.read_csv(ratings, usecols=["userId", "movieId", "rating"], dtype={"userId": "int32", "movieId": "int32", "rating": "float32"})

### EDA

#### Info and describe

In [6]:
df_movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [7]:
df_ratings.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [8]:
df_movies.info(), df_movies.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int32 
 1   title    58098 non-null  object
dtypes: int32(1), object(1)
memory usage: 681.0+ KB


(None,
              movieId
 count   58098.000000
 mean   111919.516197
 std     59862.660956
 min         1.000000
 25%     72437.750000
 50%    126549.000000
 75%    161449.500000
 max    193886.000000)

In [9]:
df_ratings.info(), df_ratings.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int32  
 1   movieId  int32  
 2   rating   float32
dtypes: float32(1), int32(2)
memory usage: 317.6 MB


(None,
              userId       movieId        rating
 count  2.775344e+07  2.775344e+07  2.775344e+07
 mean   1.419420e+05  1.848800e+04  3.530446e+00
 std    8.170740e+04  3.510263e+04  1.066353e+00
 min    1.000000e+00  1.000000e+00  5.000000e-01
 25%    7.117600e+04  1.097000e+03  3.000000e+00
 50%    1.420220e+05  2.716000e+03  3.500000e+00
 75%    2.124590e+05  7.150000e+03  4.000000e+00
 max    2.832280e+05  1.938860e+05  5.000000e+00)

#### Sorted ratings

In [17]:
ratings_sorted = df_ratings.sort_values(ascending=False, by="rating")
ratings_sorted

,userId,movieId,rating
13876722,142022,1201,5.0
3590498,36897,1408,5.0
14573661,149158,140267,5.0
3590521,36897,3793,5.0
14573665,149158,142492,5.0
...,...,...,...
11394855,117082,3975,0.5
4400473,45176,3879,0.5
23199509,237198,65261,0.5
4400477,45176,3968,0.5


In [20]:
df = ratings_sorted.groupby("movieId")["movieId"].sum().sort_values(ascending=False).head(10)
df

movieId
79132     3.282000e+09
58559     2.619988e+09
109487    2.527069e+09
134130    2.167541e+09
99114     2.026188e+09
116797    1.858941e+09
72998     1.857580e+09
134853    1.841957e+09
91529     1.804128e+09
68954     1.802664e+09
Name: movieId, dtype: float64

#### Plot for movies per year

In [13]:
## Sort out movies per decade